<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Импорты" data-toc-modified-id="Импорты-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Импорты</a></span></li><li><span><a href="#Загрузка-данных" data-toc-modified-id="Загрузка-данных-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Загрузка данных</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Подготовка данных</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Подготовка

### Импорты

In [1]:
import pandas as pd
import re
import nltk

from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier
from sklearn.metrics import f1_score, confusion_matrix
from nltk.corpus import stopwords as nltk_stopwords

import warnings
warnings.filterwarnings("ignore")

In [2]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Загрузка данных

In [3]:
data = pd.read_csv('/datasets/toxic_comments.csv')

In [4]:
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


Удалим столбец, повторяющий индекс

In [6]:
data = data.drop(columns=['Unnamed: 0'])

Сразу проверим соотношение классов. Ответов класса 0 существенно больше, чем 1. В дальнейшем необходимо будет сбалансировать эту ситуацию.

In [7]:
data['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

### Подготовка данных

Напишем две функции, которые лемматизируют текст и очистят его от регулярных выражений.

In [8]:
import sys

In [9]:
!{sys.executable} -m pip install spacy
!{sys.executable} -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     |████████████████████████████████| 13.9 MB 2.2 MB/s eta 0:00:01.6 MB 2.2 MB/s eta 0:00:03
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [10]:
import spacy

In [11]:
nlp = spacy.load('en_core_web_sm')

In [12]:
def clear_text(text):
    re_text = re.sub(r'[^a-zA-Z]', ' ', text)
    re_text = " ".join(re_text.split())
    
    return re_text

In [13]:
def lemmatize(text):
    doc = nlp(text)
    lemm_text = " ".join(token.lemma_ for token in doc)
    
    return lemm_text

In [14]:
%%time

data['lemm_text'] = data['text'].apply(lambda x: lemmatize(clear_text(x)))

CPU times: user 43min 21s, sys: 8.37 s, total: 43min 30s
Wall time: 43min 33s


Переведем весь текст в нижний регистр.

In [15]:
data['lemm_text'] = data['lemm_text'].str.lower()

In [16]:
data.head()

,text,toxic,lemm_text
0,Explanation\nWhy the edits made under my usern...,0,explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,d aww he match this background colour i m seem...
2,"Hey man, I'm really not trying to edit war. It...",0,hey man i m really not try to edit war it s ju...
3,"""\nMore\nI can't make any real suggestions on ...",0,more i can t make any real suggestion on impro...
4,"You, sir, are my hero. Any chance you remember...",0,you sir be my hero any chance you remember wha...


## Обучение

Лемматизированный текст будет составлять признаки, на которых будет обучаться модель.

In [17]:
features = data['lemm_text']
target = data['toxic']

Разделим датасет на обучающую и тестовую выборки в соотношении 0.25.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=12345, test_size=0.25)

Применим метод TF-IDF для учета частоты употребления слов. Также почистим текст от слов без смысловой нагрузки.

In [19]:
count_tf_idf = TfidfVectorizer(stop_words=stopwords)

In [20]:
tf_idf_train = count_tf_idf.fit_transform(X_train)

In [21]:
tf_idf_test = count_tf_idf.transform(X_test)

Посчитаем результат на трех моделях.

In [22]:
lr = LogisticRegression(random_state=12345)

lr_grid_space = {'max_iter':[1, 200],
                 'class_weight': ['balanced'],
                 'solver': ['liblinear']
                }

In [23]:
dtc = DecisionTreeClassifier(random_state=12345)

dtc_grid_space = {'max_depth':[1, 10],
                  'min_samples_leaf':[2,5],
                  'class_weight': ['balanced']
                 }

In [24]:
rfc = RandomForestClassifier(random_state=12345)

rfc_grid_space = {'n_estimators':[1, 100],
                  'max_depth':[1, 10],
                  'class_weight': ['balanced']
                 }

In [26]:
def find_model(model, grid_space):
    grid = GridSearchCV(model, param_grid=grid_space, cv=5, n_jobs=-1, scoring='f1', verbose=False)
    model_grid = grid.fit(tf_idf_train, y_train)
    print(model_grid.best_score_)
    print(model_grid.best_params_)
    return model_grid

In [27]:
%%time

lr_model = find_model(lr, lr_grid_space)
lr_model

0.7485373556893962
{'class_weight': 'balanced', 'max_iter': 200, 'solver': 'liblinear'}
CPU times: user 40.4 s, sys: 42.8 s, total: 1min 23s
Wall time: 1min 23s


GridSearchCV(cv=5, estimator=LogisticRegression(random_state=12345), n_jobs=-1,
             param_grid={'class_weight': ['balanced'], 'max_iter': [1, 200],
                         'solver': ['liblinear']},
             scoring='f1', verbose=False)

In [28]:
%%time

dtc_model = find_model(dtc, dtc_grid_space)
dtc_model

0.5650264251080814
{'class_weight': 'balanced', 'max_depth': 10, 'min_samples_leaf': 5}
CPU times: user 4min 31s, sys: 850 ms, total: 4min 32s
Wall time: 4min 32s


GridSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=12345),
             n_jobs=-1,
             param_grid={'class_weight': ['balanced'], 'max_depth': [1, 10],
                         'min_samples_leaf': [2, 5]},
             scoring='f1', verbose=False)

In [29]:
%%time

rfc_model = find_model(rfc, rfc_grid_space)
rfc_model

0.368292852131008
{'class_weight': 'balanced', 'max_depth': 10, 'n_estimators': 100}
CPU times: user 2min 16s, sys: 623 ms, total: 2min 17s
Wall time: 2min 17s


GridSearchCV(cv=5, estimator=RandomForestClassifier(random_state=12345),
             n_jobs=-1,
             param_grid={'class_weight': ['balanced'], 'max_depth': [1, 10],
                         'n_estimators': [1, 100]},
             scoring='f1', verbose=False)

Самый высокий результат показала модель логистической регрессии. Он близок к искомому 0.75. Проверим ее качество на тестовой выборке.

In [31]:
result = pd.DataFrame ([
    [lr_model.best_score_],
    [dtc_model.best_score_],
    [rfc_model.best_score_]], 
    columns=['f1'],
    index=['LogisticRegression', 'DecisionTree','RandomForest'])

result.sort_values(by='f1', ascending=False)

,f1
LogisticRegression,0.748537
DecisionTree,0.565026
RandomForest,0.368293


Тестовая выборка показала результат метрики F1 выше 0.75.

In [32]:
model = LogisticRegression(random_state=12345, class_weight='balanced', max_iter=200, solver='liblinear')
model.fit(tf_idf_train, y_train)
predictions = model.predict(tf_idf_test)
f1 = f1_score(y_test, predictions)
f1

0.753610691959474

Построим матрицу ошибок. Чаще всего у нашей модели возникали сложности с предсказанием отрицательных ответов.

In [33]:
confusion_matrix(y_test, predictions)

array([[34041,  1738],
       [  548,  3496]])

## Выводы

Наша задача заключалась в написании модели, способной классифицировать комментарии к товарам в интернет-магазине на позитивные и негативные. 


В нашем распоряжении был набор данных с разметкой токсичности комментариев. По итогам работы метрика F1 модели должна быть не меньше 0.75.


В ходе работы мы лемматризировали текст, очистили его от регулярных выражений и слов без смысловой нагрузки, а также применили метод TF-IDF для учета частоты употребления слов.


В моделях обучения в качестве одного из гиперпараметров был применен баланс классов, так как в исходных данных был сильный перевес в пользу класса 0. 


Самый высокий результат показала модель логистической регрессии. Модель допустила заметное количество ошибок при определении ответов класса 0. Тем не менее, на тестовой выборке удалось получить метрику F1 на уровне 0.753.